In [372]:
import pandas as pd
import numpy as np

df_profiles = pd.read_csv("student_or_staff_profiles.csv")
df_profiles['student_id/staff_id'] = df_profiles['student_id'].fillna(df_profiles['staff_id'])
df_profiles = df_profiles[['entity_id','name','role','email','department','student_id/staff_id','card_id','device_hash','face_id']]
df_swipes = pd.read_csv("campus_card_swipes.csv")
df_wifi = pd.read_csv("wifi_associations_logs.csv")
df_wifi['timestamp']=pd.to_datetime(df_wifi['timestamp'])


In [373]:
swipes_with_person = pd.merge(df_swipes[['card_id','location_id','timestamp']], df_profiles[['card_id', 'student_id/staff_id', 'name','role','entity_id','email','department','face_id']], on='card_id', how='left')
wifi_with_person = pd.merge(df_wifi[['device_hash','ap_id','timestamp',]], df_profiles[['device_hash', 'student_id/staff_id', 'name','role','entity_id','email','department','face_id']], on='device_hash', how='left')

In [374]:
swipes_std = swipes_with_person.copy()
swipes_std.rename(columns={'location_id': 'location_id/ap_id'}, inplace=True)
swipes_std.rename(columns={'card_id': 'card_id/device_hash'}, inplace=True)
swipes_std['event_type'] = 'card_swipe'

wifi_std = wifi_with_person.copy()
wifi_std.rename(columns={'ap_id': 'location_id/ap_id'}, inplace=True)
wifi_std.rename(columns={'device_hash': 'card_id/device_hash'}, inplace=True)
wifi_std['event_type'] = 'wifi_log'


In [375]:
unified_timeline_swipe_wifi = pd.concat([swipes_std, wifi_std], ignore_index=True)
unified_timeline_swipe_wifi.sort_values(by=['student_id/staff_id', 'timestamp'], inplace=True)
unified_timeline_swipe_wifi.reset_index(drop=True, inplace=True)
#unified_timeline_swipe_wifi

In [376]:
#unified_timeline_swipe_wifi.to_csv("unified_timeline_swipe_wifi.csv")

In [377]:
df_checkouts = pd.read_csv("library_checkouts.csv")

checkouts_with_person = pd.merge(df_checkouts, df_profiles[['card_id', 'student_id/staff_id', 'name','role','entity_id','email','department','face_id']], on = 'entity_id', how = 'left')
checkouts_with_person['checkout_id(book_id)'] = checkouts_with_person['checkout_id']+"("+checkouts_with_person['book_id']+")"

checkouts_std = checkouts_with_person[['checkout_id(book_id)','timestamp','student_id/staff_id','name','entity_id','role','email','department','face_id']]
checkouts_std['location_id/ap_id']='LIBRARY'
checkouts_std.rename(columns={'checkout_id(book_id)': 'checkout_id(book_id)/card_id/device_hash'}, inplace=True)
checkouts_std['event_type'] = 'library_checkout'
checkouts_std = checkouts_std[['checkout_id(book_id)/card_id/device_hash','location_id/ap_id','timestamp','student_id/staff_id','name','role','entity_id','event_type','email','department','face_id']]
#checkouts_std

C:\Users\anubh\AppData\Local\Temp\ipykernel_17856\1485214148.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checkouts_std['location_id/ap_id']='LIBRARY'
C:\Users\anubh\AppData\Local\Temp\ipykernel_17856\1485214148.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checkouts_std.rename(columns={'checkout_id(book_id)': 'checkout_id(book_id)/card_id/device_hash'}, inplace=True)
C:\Users\anubh\AppData\Local\Temp\ipykernel_17856\1485214148.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [378]:
unified_timeline_swipe_wifi.rename(columns={'card_id/device_hash': 'checkout_id(book_id)/card_id/device_hash'}, inplace=True)
unified_timeline_swipe_wifi_checkouts = pd.concat([unified_timeline_swipe_wifi, checkouts_std], ignore_index=True)
unified_timeline_swipe_wifi_checkouts.sort_values(by=['student_id/staff_id', 'timestamp'], inplace=True)
unified_timeline_swipe_wifi_checkouts.reset_index(drop=True, inplace=True)
unified_timeline_swipe_wifi_checkouts.rename(columns={'location_id/ap_id': 'location_id/ap_id/room_id'}, inplace=True)
#unified_timeline_swipe_wifi_checkouts

In [379]:
#unified_timeline_swipe_wifi_checkouts.to_csv("unified_timeline_swipe_wifi_checkouts.csv")

In [380]:
df_lab = pd.read_csv("lab_bookings.csv")
df_lab['start_time'] = pd.to_datetime(df_lab['start_time'])
df_lab['end_time'] = pd.to_datetime(df_lab['end_time'])
df_lab = df_lab[df_lab['attended (YES/NO)']=='YES']
df_lab.reset_index(drop=True, inplace=True)
df_lab = df_lab[['booking_id','entity_id','room_id','start_time','end_time']]
df_lab.rename(columns={'booking_id':'booking_id/checkout_id(book_id)/card_id/device_hash'}, inplace = True)

df_lab_start = df_lab[['booking_id/checkout_id(book_id)/card_id/device_hash','entity_id','room_id','start_time']]
df_lab_end = df_lab[['booking_id/checkout_id(book_id)/card_id/device_hash','entity_id','room_id','end_time']]

df_lab_start.rename(columns={'room_id':'location_id/ap_id/room_id'}, inplace = True)
df_lab_end.rename(columns={'room_id':'location_id/ap_id/room_id'}, inplace = True)

df_lab_start.rename(columns={'start_time':'timestamp'}, inplace = True)
df_lab_end.rename(columns={'end_time':'timestamp'}, inplace = True)

df_lab_start['event_type']='lab_start'
df_lab_end['event_type']='lab_end'

labs_with_person_start = pd.merge(df_lab_start, df_profiles, on = 'entity_id', how = 'left')
labs_with_person_end = pd.merge(df_lab_end, df_profiles, on = 'entity_id', how = 'left')

labs_with_person_start_std = labs_with_person_start[['booking_id/checkout_id(book_id)/card_id/device_hash','location_id/ap_id/room_id','timestamp','student_id/staff_id','name','role','entity_id','event_type','email','department','face_id']]
labs_with_person_end_std = labs_with_person_end[['booking_id/checkout_id(book_id)/card_id/device_hash','location_id/ap_id/room_id','timestamp','student_id/staff_id','name','role','entity_id','event_type','email','department','face_id']]

labs_with_person_std = pd.concat([labs_with_person_start_std, labs_with_person_end_std], ignore_index=True)
#labs_with_person_std

C:\Users\anubh\AppData\Local\Temp\ipykernel_17856\101524182.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lab_start.rename(columns={'room_id':'location_id/ap_id/room_id'}, inplace = True)
C:\Users\anubh\AppData\Local\Temp\ipykernel_17856\101524182.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lab_end.rename(columns={'room_id':'location_id/ap_id/room_id'}, inplace = True)
C:\Users\anubh\AppData\Local\Temp\ipykernel_17856\101524182.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [381]:
unified_timeline_swipe_wifi_checkouts.rename(columns={'checkout_id(book_id)/card_id/device_hash': 'booking_id/checkout_id(book_id)/card_id/device_hash'}, inplace=True)
unified_timeline_swipe_wifi_checkouts_labs = pd.concat([unified_timeline_swipe_wifi_checkouts, labs_with_person_std ], ignore_index=True)
unified_timeline_swipe_wifi_checkouts_labs.sort_values(by=['student_id/staff_id', 'timestamp'], inplace=True)
unified_timeline_swipe_wifi_checkouts_labs.reset_index(drop=True, inplace=True)
unified_timeline_swipe_wifi_checkouts_labs = unified_timeline_swipe_wifi_checkouts_labs[['student_id/staff_id','timestamp','name','role','department','email','booking_id/checkout_id(book_id)/card_id/device_hash','location_id/ap_id/room_id','entity_id','face_id','event_type']]

In [382]:
unified_timeline_swipe_wifi_checkouts_labs.to_csv("unified_timeline_swipe_wifi_checkouts_labs.csv")